# Quantum Key Distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import random
from qiskit import execute, Aer, IBMQ, QuantumCircuit
from qiskit.tools.jupyter import *
from qiskit.visualization import *

from QKD_helpers import *

In [ ]:
# Configuring account
backend = Aer.get_backend('qasm_simulator')

## Basics

Make a single qubit circuit with the qubit in the $|0\rangle$ state

In [ ]:
qc = QuantumCircuit(1,1) # First index determines the number of qubits
                         # Second index determines the number of classical bits (useful for measurement)
qc.reset(0)
qc.draw()

Make a single qubit circuit with the qubit in the $|1\rangle$ state

In [ ]:
qc = QuantumCircuit(1,1)
qc.reset(0)
### ADD Code here
qc.draw()

Make a single qubit circuit with the qubit in the $|+\rangle$ state

In [ ]:
qc = QuantumCircuit(1,1)
qc.reset(0)
### ADD Code here
qc.draw()

Make a single qubit circuit with the qubit in the $|-\rangle$ state

In [ ]:
qc = QuantumCircuit(1,1)
qc.reset(0)
### ADD Code here
qc.draw()

Do a single measurement of a qubit in the $|1\rangle$ state

In [ ]:
qc = QuantumCircuit(1,1)
qc.x(0)
qc.measure(0,0)
print(qc)

results = execute(qc, backend=backend, shots=1).result()
print('measurement result: ',results.get_counts())

## QKD

### Alice

In [ ]:
bitstring_length=500

class Alice():
    
    def __init__(self):
        self.prepared_bitstring=''
        self.bases=''
        self.key=''
        
    def send_qubits(self):
        # Generate the bitsrting that Alice will encode
        self.bitstring=str('{0:0100b}'.format(random.getrandbits(bitstring_length)))
        
        # Generate the basis that Alice uses to encode her bitstring. 
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        self.basis=str('{0:0100b}'.format(random.getrandbits(bitstring_length)))
        
        qubits=[]
        # Hint: For every qubit make a seperate circuit, apply the proper gates and then append it to this list
        
        #
        #
        # Insert your code here
        #
        #
        #
        
        return qubits
    
    def construct_key_from_indecies(indecies):
        for idx in indecies:
            self.key+= self.prepared_bitstring[idx]

### Bob

In [ ]:
class Bob():
    def __init__(self):
        self.measured_bitstring=''
        self.bases=''
        self.key=''
        
    def measure_qubits(self,qubits):
        # Generate the basis that Bob uses to measure the qubits. 
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        self.basis=str('{0:0100b}'.format(random.getrandbits(bitstring_length)))
        
        
        #
        #
        # Insert your code here
        #
        #
        #
    def construct_key_from_indecies(indecies):
        for idx in indecies:
            self.key+= self.measured_bitstring[idx]

In [ ]:
def compare_and_make_key(alice, bob):
    
    indecies=[]
    for i in range(bitstring_length):
        if alice.basis[i]==bob.basis[i]: indecies.append(i)
    
    alice.construct_key_from_indecies(indecies)
    bob.construct_key_from_indecies(indecies)

### Protocol

In [ ]:
# define Alice
alice= Alice()
# define Bob
bob= Bob()

# Alice prepares and sends her qubits to Bob
sent_qubits= alice.send_qubits()

# Bob measures the qubits
bob.measure_qubits(sent_qubits)

# Alice and Bob compare their basis and agree on a key
compare_and_make_key(alice, bob)

### Secure communication

In [ ]:
secret_message=generate_secret_key()

def binary_add(s,k):
    return str( (int(s)+int(k))%2 )

# Alice encodes the secret message using her key
encoded_message = ''
for i in range(len(secret_message)):
    encoded_message += #Fill in here
    
# Bob decodes the secret message using his key
decoded_message = ''
for i in range(len(encoded_message)):
    decoded_message += #Fill in here


In [ ]:
# Let's see what does the message that Bob got from Alice say: 
text_from_bits(decoded_message)